# Seminar BA & QM
### Code Bol.com Case

Below you will find the code for the Bol.com case from group 8. 

Table of contents:

*  [Data preparation](#Data-preparation)
*  [Functions](#Functions)
*  [Models](#Models)

****

In [3]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import time

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Data Preparation
#### Load data and set data-types 

In [4]:
#Import data
df_2019 = pd.read_csv('/Users/LV/Documents/ECONOMETRIE-ECO/MASTER/Seminar/2021_Case_Bol.com_Happy_Matches/data/data_2019.csv', low_memory = False)
df_2020 = pd.read_csv('/Users/LV/Documents/ECONOMETRIE-ECO/MASTER/Seminar/2021_Case_Bol.com_Happy_Matches/data/data_2020.csv', low_memory = False)

#Concat files and create new index
df_full = pd.concat([df_2019, df_2020])
df_full = df_full.reset_index(drop = True)

print('Total # records: ',df_full.shape[0])

Total # records:  4779466


In [5]:
#Rename columns
rename_columns = {'datetTimeFirstDeliveryMoment': 'dateTimeFirstDeliveryMoment',
                  'generalMatchClassification'  : 'detailedMatchClassification',
                  'detailedMatchClassification' : 'generalMatchClassification',
                  'quanityReturned'             : 'quantityReturned'}

df_full = df_full.rename(columns = rename_columns)

In [6]:
#Transform dates to date-type
df_full['orderDate']                   = pd.to_datetime(df_full['orderDate'])
df_full['cancellationDate']            = pd.to_datetime(df_full['cancellationDate'])
df_full['promisedDeliveryDate']        = pd.to_datetime(df_full['promisedDeliveryDate'])
df_full['shipmentDate']                = pd.to_datetime(df_full['shipmentDate'])
df_full['dateTimeFirstDeliveryMoment'] = pd.to_datetime(df_full['dateTimeFirstDeliveryMoment'])
df_full['startDateCase']               = pd.to_datetime(df_full['startDateCase'])
df_full['returnDateTime']              = pd.to_datetime(df_full['returnDateTime'])
df_full['registrationDateSeller']      = pd.to_datetime(df_full['registrationDateSeller'])

#Change type of columns
dtype = {'calculationDefinitive': bool,
         'noCancellation'       : bool,
         'noCase'               : bool,
         'hasOneCase'           : bool,
         'hasMoreCases'         : bool,
         'noReturn'             : bool}

df_full = df_full.astype(dtype)

#### Remove Noise

In [7]:
#Remove non-sensible rows
noise = df_full.loc[(df_full['startDateCase']        < df_full['orderDate']) | 
                   (df_full['cancellationDate']      < df_full['orderDate']) |
                   (df_full['promisedDeliveryDate']  < df_full['orderDate']) |
                   (df_full['shipmentDate']          < df_full['orderDate']) |
                   (df_full['returnDateTime']        < df_full['orderDate']) |
                   (df_full['cancellationDate']      > df_full['returnDateTime']) |
                   (df_full['shipmentDate']          > df_full['returnDateTime']) |
                   (df_full['orderDate']             < df_full['registrationDateSeller']) |
                   (df_full['orderDate']             > df_full['dateTimeFirstDeliveryMoment'].dt.normalize()) |
                   (df_full['cancellationDate']      > df_full['dateTimeFirstDeliveryMoment'].dt.normalize()) |
                   (df_full['shipmentDate']          > df_full['dateTimeFirstDeliveryMoment'].dt.normalize()) |
                   ((df_full['returnDateTime']       < df_full['dateTimeFirstDeliveryMoment'].dt.normalize()) & 
                                                       (df_full['dateTimeFirstDeliveryMoment'].notnull()) &
                                                       (df_full['returnDateTime'].notnull())) |
                   ((df_full['cancellationDate']     > df_full['shipmentDate']) &
                                                       ((df_full['cancellationReasonCode'] == 'CUST_FE') |
                                                       (df_full['cancellationReasonCode'] == 'CUST_CS'))) |
                   (df_full['registrationDateSeller'].isnull()) |
                   (df_full['promisedDeliveryDate'].isnull())].index

#Drop noise data
df = df_full.drop(index = noise)
print(len(noise), 'complete records removed from the data')
print('Cleaned # records: ',df.shape[0],'\n')

#Sort rows on orderDate and create new index
df = df.sort_values(by = 'orderDate')
df = df.reset_index(drop = True)

#Remove inconsistent values
cancellationNoise = df.loc[(df['noCancellation'] == True) & (df['cancellationDate'].notnull())].index
returnNoise       = df.loc[(df['noReturn'] == True) & (df['returnDateTime'].notnull())].index
caseNoise         = df.loc[(df['noCase'] == True) & (df['startDateCase'].notnull())].index
quantityNoise     = df.loc[df['quantityReturned'] > df['quantityOrdered']].index
deliveryNoise     = df.loc[(df['dateTimeFirstDeliveryMoment'].notnull()) & (df['onTimeDelivery'].isnull())].index

df.loc[cancellationNoise, ['cancellationDate','cancellationReasonCode']] = None
df.loc[returnNoise,       ['returnDateTime','quantityReturned','returnCode']] = None
df.loc[caseNoise,         ['startDateCase','cntDistinctCaseIds','hasOneCase','hasMoreCases']] = None
df.loc[quantityNoise,     ['quantityReturned']] = df.loc[quantityNoise, ['quantityOrdered']]
df.loc[deliveryNoise,     ['dateTimeFirstDeliveryMoment']] = None

print('# Records where cancellation values are emptied:',len(cancellationNoise))
print('# Records where return values are emptied: \t',len(returnNoise))
print('# Records where case values are emptied: \t',len(caseNoise))
print('# Records where quantity values are equalized: \t',len(quantityNoise))
print('# Records where delivery values are emptied: \t',len(deliveryNoise))

6516 complete records removed from the data
Cleaned # records:  4772950 

# Records where cancellation values are emptied: 53780
# Records where return values are emptied: 	 8208
# Records where case values are emptied: 	 0
# Records where quantity values are equalized: 	 14722
# Records where delivery values are emptied: 	 4103


#### Create New Variables

##### Time Related

In [8]:
#Create new variables related to periods (days difference)
df['caseDays']             = (df['startDateCase'] - df['orderDate']).dt.days
df['returnDays']           = (df['returnDateTime'] - df['orderDate']).dt.days
df['cancellationDays']     = (df['cancellationDate'] - df['orderDate']).dt.days
df['actualDeliveryDays']   = (df['dateTimeFirstDeliveryMoment'].dt.normalize() - df['orderDate']).dt.days
df['shipmentDays']         = (df['shipmentDate'] - df['orderDate']).dt.days
df['partnerSellingMonths'] = (df['orderDate'] - df['registrationDateSeller']).dt.days
df['promisedDeliveryDays'] = (df['promisedDeliveryDate'] - df['orderDate']).dt.days

#Time related variables
df['orderYear']    = df['orderDate'].dt.year
df['orderMonth']   = df['orderDate'].dt.month
df['orderWeekday'] = df['orderDate'].dt.weekday
df['orderCorona']  = df['orderDate'].apply(lambda x: True if x > datetime.strptime('2020-03-20','%Y-%m-%d') else False)

#Create dummy variables for weekdays and months
df['orderMonday']    = df['orderWeekday'].apply(lambda x: True if x == 1 else False)
df['orderTuesday']   = df['orderWeekday'].apply(lambda x: True if x == 2 else False)
df['orderWednesday'] = df['orderWeekday'].apply(lambda x: True if x == 3 else False)
df['orderThursday']  = df['orderWeekday'].apply(lambda x: True if x == 4 else False)
df['orderFriday']    = df['orderWeekday'].apply(lambda x: True if x == 5 else False)
df['orderSaturday']  = df['orderWeekday'].apply(lambda x: True if x == 6 else False)
df['orderSunday']    = df['orderWeekday'].apply(lambda x: True if x == 7 else False)

df['orderJanuary']   = df['orderMonth'].apply(lambda x: True if x == 1 else False)
df['orderFebruary']  = df['orderMonth'].apply(lambda x: True if x == 2 else False)
df['orderMarch']     = df['orderMonth'].apply(lambda x: True if x == 3 else False)
df['orderApril']     = df['orderMonth'].apply(lambda x: True if x == 4 else False)
df['orderMay']       = df['orderMonth'].apply(lambda x: True if x == 5 else False)
df['orderJune']      = df['orderMonth'].apply(lambda x: True if x == 6 else False)
df['orderJuly']      = df['orderMonth'].apply(lambda x: True if x == 7 else False)
df['orderAugust']    = df['orderMonth'].apply(lambda x: True if x == 8 else False)
df['orderSeptember'] = df['orderMonth'].apply(lambda x: True if x == 9 else False)
df['orderOctober']   = df['orderMonth'].apply(lambda x: True if x == 10 else False)
df['orderNovember']  = df['orderMonth'].apply(lambda x: True if x == 11 else False)
df['orderDecember']  = df['orderMonth'].apply(lambda x: True if x == 12 else False)

##### Other

In [9]:
df['productTitleLength'] = len(df['productTitle'])

df['fulfilmentByBol'] = df['fulfilmentType'].apply(lambda x: True if x == 'FBB' else False)

df['countryCodeNL']   = df['countryCode'].apply(lambda x: True if x == 'NL' else False)
df['countryOriginNL'] = df['countryOriginSeller'].apply(lambda x: True if x == 'NL' else False)
df['countryOriginBE'] = df['countryOriginSeller'].apply(lambda x: True if x == 'BE' else False)
df['countryOriginDE'] = df['countryOriginSeller'].apply(lambda x: True if x == 'DE' else False)

##### Determinant Classification

In [10]:
def determinantClassification(df):
    """
    Function to create column with determinants for each order.
    """
    if (df.noCancellation == 1 and df.noReturn == 1 and df.noCase == 1 and df.onTimeDelivery == True): 
        return 'All good'
    elif (df.noCancellation == 1 and df.noReturn == 1 and df.noCase == 1 and df.onTimeDelivery == None):
        return 'Unknown delivery'
    elif (df.noCancellation == 1 and df.noReturn == 1 and df.noCase == 1 and df.onTimeDelivery == False):
        return 'Late delivery'
    elif (df.noCancellation == 1 and df.noReturn == 1 and df.noCase == 0 and df.onTimeDelivery == True):
        return 'Case'
    elif (df.noCancellation == 1 and df.noReturn == 1 and df.noCase == 0 and df.onTimeDelivery == None):
        return 'Case + Unknown delivery'
    elif (df.noCancellation == 1 and df.noReturn == 1 and df.noCase == 0 and df.onTimeDelivery == False):
        return 'Case + Late delivery'
    elif (df.noCancellation == 1 and df.noReturn == 0 and df.noCase == 1 and df.onTimeDelivery == True):
        return 'Return'
    elif (df.noCancellation == 1 and df.noReturn == 0 and df.noCase == 1 and df.onTimeDelivery == None):
        return 'Return + Unknown delivery'
    elif (df.noCancellation == 1 and df.noReturn == 0 and df.noCase == 1 and df.onTimeDelivery == False):
        return 'Return + Late delivery'
    elif (df.noCancellation == 1 and df.noReturn == 0 and df.noCase == 0 and df.onTimeDelivery == True):
        return 'Return + Case'
    elif (df.noCancellation == 1 and df.noReturn == 0 and df.noCase == 0 and df.onTimeDelivery == None):
        return 'Return + Case + Unknown delivery'
    elif (df.noCancellation == 1 and df.noReturn == 0 and df.noCase == 0 and df.onTimeDelivery == False):
        return 'Return + Case + Late delivery'
    elif (df.noCancellation == 0 and df.noReturn == 1 and df.noCase == 0 and df.onTimeDelivery == True):
        return 'Cancellation + Case'
    elif (df.noCancellation == 0):
        return 'Cancellation'

In [11]:
df['determinantClassification'] = df.apply(determinantClassification, axis = 1)
df['determinantClassification'].value_counts()

All good                         2685606
Late delivery                     146725
Return                            145556
Case                               42643
Return + Case                      24204
Cancellation                       23690
Return + Late delivery              8426
Case + Late delivery                8352
Return + Case + Late delivery       2134
Cancellation + Case                    3
Name: determinantClassification, dtype: int64

##### Transporter

In [12]:
def transporterCluster(transporterCode):
    """
    Function to create a new manually clustered transporter variable: 28 -> 5 categories
    """
    if transporterCode in ['AH-NL','TNT','TNT-EXPRESS','TNT-EXTRA']:
        return 'POSTNL'
    elif transporterCode in ['DHL','DHL_DE','DHLFORYOU']:
        return 'DHL'
    elif transporterCode in ['DPD-NL','DPD-BE']:
        return 'DPD'
    elif transporterCode in ['BRIEFPOST','BPOST_BE','BPOST_BRIEF','DHL-GLOBAL-MAIL','TNT_BRIEF']:
        return 'BRIEF'
    else:
        return 'OTHER'

In [13]:
df['transporterCodeGeneral'] = df['transporterCode'].apply(transporterCluster)
df['transporterCodeGeneral'].value_counts()

POSTNL    2110753
BRIEF     1576462
DHL        436975
DPD        329746
OTHER      319014
Name: transporterCodeGeneral, dtype: int64

##### Product Group

In [14]:
def productGroupCluster(productGroup):
    """
    Function to create a new manually clustered product group variable based on categories bol.com
    60 -> 14 groups.
    """
    if productGroup in ['Dutch Books PG','Ebooks and Audiobooks','International Books PG']:
        return 'Books'
    elif productGroup in ['Games Accessories','Games Consoles','Games Software Physical',
                          'Movies','Music']:
        return 'Music, Film & Games'
    elif productGroup in ['Camera','Desktop Monitor and Beamer','Ereaders and Accessories',
                          'Laptop Computers','PC Accessories','Personal Audio',
                          'Sound and Vision Accessories','Storage and Network',
                          'Telephone and Tablet Accessories','Telephones and Tablets','Television']:
        return 'Computer & Electronics'
    elif productGroup in ['General Toys','Recreational and Outdoor Toys']:
        return 'Toys & Hobby'
    elif productGroup in ['Baby and Kids Fashion','Baby PG']:
        return 'Baby & Kids'
    elif productGroup in ['Daily Care PG','Health PG','Perfumery PG','Personal Care']:
        return 'Health & Care'
    elif productGroup in ['Footwear','Jewelry and Watches','Mens and Womens Fashion','Wearables']:
        return 'Fashion, Shoes & Accessories'
    elif productGroup in ['Bodyfashion and Beachwear','Camping and Outdoor','Cycling',
                          'Sporting Equipment','Sportswear','Travel Bags and Accessories']:
        return 'Sports, Outdoor & Travel'
    elif productGroup in ['Educational Dutch','Educational International','Printing and Ink']:
        return 'Office & School'
    elif productGroup in ['Supermarket PG'] :
        return 'Food & Beverage'
    elif productGroup in ['Furniture','Heating and Air','Home Decoration','Home Entertainment',
                          'Household','Household Appliances','Kitchen','Kitchen Machines',
                          'Lighting','Major Domestic Appliances PG','Plumbing and Safety']:
        return 'Home, Cooking & Household'
    elif productGroup in ['Garden','Pet PG','Textiles','Tools and Paint']:
        return 'Pets, Garden & Jobs'
    elif productGroup in ['Car and Motorcycle'] :
        return 'Car & Motor'
    else:
        return 'Other'

In [15]:
df['productGroupGeneral'] = df['productGroup'].apply(productGroupCluster)
df['productGroupGeneral'].value_counts()

Computer & Electronics          1387679
Home, Cooking & Household        797874
Sports, Outdoor & Travel         522098
Toys & Hobby                     500977
Pets, Garden & Jobs              339813
Health & Care                    299049
Food & Beverage                  258769
Books                            184581
Music, Film & Games              163842
Baby & Kids                      113707
Fashion, Shoes & Accessories     110067
Office & School                   52270
Car & Motor                       29753
Other                             12471
Name: productGroupGeneral, dtype: int64

In [16]:
#Create dummies of new product grouping
for group in df['productGroupGeneral'].unique():
    
    columnName = 'group' + group.split(' ')[0].replace(',','')
    df[columnName] = df['productGroupGeneral'].apply(lambda x: True if x == group else False)

##### Total Columns

In [17]:
print(df.columns)
print('Total: ',len(df.columns),' columns')

Index(['orderDate', 'productId', 'sellerId', 'totalPrice', 'quantityOrdered',
       'countryCode', 'cancellationDate', 'cancellationReasonCode',
       'promisedDeliveryDate', 'shipmentDate', 'transporterCode',
       'transporterName', 'transporterNameOther',
       'dateTimeFirstDeliveryMoment', 'fulfilmentType', 'startDateCase',
       'cntDistinctCaseIds', 'returnDateTime', 'quantityReturned',
       'returnCode', 'productTitle', 'brickName', 'chunkName', 'productGroup',
       'productSubGroup', 'productSubSubGroup', 'registrationDateSeller',
       'countryOriginSeller', 'currentCountryAvailabilitySeller',
       'calculationDefinitive', 'noCancellation', 'onTimeDelivery', 'noCase',
       'hasOneCase', 'hasMoreCases', 'noReturn', 'detailedMatchClassification',
       'generalMatchClassification', 'caseDays', 'returnDays',
       'cancellationDays', 'actualDeliveryDays', 'shipmentDays',
       'partnerSellingMonths', 'promisedDeliveryDays', 'orderYear',
       'orderMonth', 'ord

In [143]:
#Fixed Columns:
BASIC = ['totalPrice','quantityOrdered','fulfilmentByBol','countryCodeNL','countryOriginNL','countryOriginBE',
        'countryOriginDE','productTitleLength']
WEEK = ['orderMonday','orderTuesday','orderWednesday','orderThursday','orderFriday','orderSaturday','orderSunday']
MONTH = ['orderJanuary','orderFebruary','orderMarch','orderApril','orderMay','orderJune',
         'orderJuly','orderAugust','orderSeptember','orderOctober','orderNovember','orderDecember']
GROUP = ['groupHealth','groupHome','groupSports','groupComputer','groupPets','groupToys','groupBooks', 
         'groupBaby', 'groupMusic', 'groupFood','groupOffice','groupFashion','groupOther','groupCar']

#Dynamic Columns:
TRANSPORTERX = ['transporterPOSTNL/X','transporterDHL/X','transporterDPD/X','transporterBRIEF/X','transporterOTHER/X']
KNOWNX = ['caseKnownX','returnKnownX','cancellationKnownX','onTimeDeliveryKnownX','lateDeliveryKnownX']
PRODUCTX = ['productOrderCountX','productTotalCountX','productTotalReturnedX','productReturnFractionX']
SELLERX = ['sellerDailyOrdersX']

In [19]:
# Convert to CSV File
# df.to_csv('/Users/LV/Desktop/data_bol_complete.csv',index = False)

### Functions

In [43]:
def addKnownColumns(df,X):
    """
    Function to create columns which indicate whether determinants are known after X days.
    Input: X = number of days after order date at which the prediction is made
           df = dataFrame
    """
    df_ = df[['actualDeliveryDays','onTimeDelivery','shipmentDays','transporterCodeGeneral']]
    
    df['caseKnownX']           = df['caseDays'].apply(lambda x: True if x <= X else False)
    df['returnKnownX']         = df['returnDays'].apply(lambda x: True if x <= X else False)
    df['cancellationKnownX']   = df['cancellationDays'].apply(lambda x: True if x <= X else False)
    
    df_['actualDeliveryKnown'] = df['actualDeliveryDays'].apply(lambda x: True if x <= X else False)
    df_['shipmentDaysKnown'] = df['shipmentDays'].apply(lambda x: True if x <= X else False)
    
    df['onTimeDeliveryKnownX'] = df_.apply(lambda row: True if ((row.actualDeliveryKnown == True) and (row.onTimeDelivery == True)) else False, axis = 1)
    df['lateDeliveryKnownX']   = df_.apply(lambda row: True if ((row.actualDeliveryKnown == True) and (row.onTimeDelivery == False)) else False, axis = 1)
    
    for transporter in df_['transporterCodeGeneral'].unique():
        dummyColumn = 'transporter' + transporter +'/X'
        df[dummyColumn] = df_.apply(lambda row: True if ((row.shipmentDays == True) and (row.transporterCodeGeneral == transporter)) else False, axis = 1)

    return df

In [45]:
addKnownColumns(df.iloc[:100000],1)

<ipython-input-43-a1e0a7089134>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['caseKnownX']           = df['caseDays'].apply(lambda x: True if x <= X else False)
<ipython-input-43-a1e0a7089134>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['returnKnownX']         = df['returnDays'].apply(lambda x: True if x <= X else False)
<ipython-input-43-a1e0a7089134>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

1
2
3


<ipython-input-43-a1e0a7089134>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['onTimeDeliveryKnownX'] = df_.apply(lambda row: True if ((row.actualDeliveryKnown == True) and (row.onTimeDelivery == True)) else False, axis = 1)


4


<ipython-input-43-a1e0a7089134>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lateDeliveryKnownX']   = df_.apply(lambda row: True if ((row.actualDeliveryKnown == True) and (row.onTimeDelivery == False)) else False, axis = 1)


5


<ipython-input-43-a1e0a7089134>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[dummyColumn] = df_.apply(lambda row: True if ((row.shipmentDays == True) and (row.transporterCodeGeneral == transporter)) else False, axis = 1)


,orderDate,productId,sellerId,totalPrice,quantityOrdered,countryCode,cancellationDate,cancellationReasonCode,promisedDeliveryDate,shipmentDate,...,caseKnownX,returnKnownX,cancellationKnownX,onTimeDeliveryKnownX,lateDeliveryKnownX,transporterBRIEF/X,transporterDPD/X,transporterOTHER/X,transporterPOSTNL/X,transporterDHL/X
0,2019-01-01,9200000103390344,1244284,4.95,1,NL,NaT,NaN,2019-01-04,2019-01-02,...,False,False,False,False,False,True,False,False,False,False
1,2019-01-01,9200000065456100,1167286,158.00,1,NL,NaT,NaN,2019-01-07,2019-01-02,...,False,False,False,False,False,False,True,False,False,False
2,2019-01-01,9200000024539481,888610,30.00,1,BE,NaT,NaN,2019-01-03,2019-01-02,...,False,False,False,False,False,True,False,False,False,False
3,2019-01-01,9200000056338594,1308208,32.20,2,NL,2019-01-02,SELLER_UNDELIV,2019-01-03,NaT,...,False,False,True,False,False,False,False,False,False,False
4,2019-01-01,9200000085152942,1274000,24.50,1,NL,NaT,NaN,2019-01-03,2019-01-02,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2019-01-23,9200000097186560,1277546,128.85,1,BE,NaT,NaN,2019-01-24,2019-01-23,...,False,False,False,True,False,False,False,False,False,False
99996,2019-01-23,9200000096832756,1183757,249.99,1,BE,NaT,NaN,2019-02-04,2019-02-04,...,False,False,False,False,False,False,False,False,False,False
99997,2019-01-23,1000004000052943,393586,5.95,1,NL,NaT,NaN,2019-01-25,2019-01-24,...,False,False,False,False,False,True,False,False,False,False
99998,2019-01-23,9200000040296291,720508,42.99,1,BE,NaT,NaN,2019-01-24,2019-01-23,...,False,False,False,False,False,False,False,False,False,False


In [42]:
def addProductColumns(df,X):
    
    if ['productOrderCount0','productTotalCount0','productTotalReturned0','productReturnFraction0'] not in list(df.columns):
    
        df = addProductColumns0(df)
    
    if X > 0:
        
        df = addProductColumnsX(df,X)
        
    else:
        
        df['productOrderCountX'] = df['productOrderCount0']
        df['productTotalCountX'] = df['productTotalCount0']
        df['productTotalReturnedX'] = df['productTotalReturned0']
        df['productReturnFractionX'] = df['productReturnFraction0']
        
    return df

In [33]:
def addSellerColumns(df,X):
    
    if 'sellerDailyOrders0' not in list(df.columns):
    
        df = addSellerColumns0(df)
    
    if X > 0:
        
        df = addSellerColumnsX(df,X)
    
    else:
        
        df['sellerDailyOrdersX'] = df['sellerDailyOrders0']
        
    return df

In [34]:
def addProductColumns0(df): 
    """
    Function to add 4 columns: productOrderCount, productTotalCount, productTotalReturned and productReturnFraction.
    Input: dataFrame with columns: 'productId','orderDate','quantityOrdered','quantityReturned','returnDateTime'.
    """
    df = df.sort_values(by = ['productId','orderDate'])
    df = df.reset_index(drop = True)
    
    df_ = df[['productId','orderDate','quantityOrdered','quantityReturned','returnDateTime']]
    
    #ProductTotalCount
    pivot = df_.groupby(['productId','orderDate']).quantityOrdered.sum().groupby('productId').cumsum()
    productTotalCount = df_.merge(pivot, 
                                left_on=['productId','orderDate'], 
                                right_index=True, 
                                how = 'left').quantityOrdered_y
    
    #ProductOrderCount
    pivot = df_.groupby(['productId','orderDate']).quantityOrdered.count().groupby('productId').cumsum()
    productOrderCount = df_.merge(pivot, 
                                left_on=['productId','orderDate'], 
                                right_index=True, 
                                how = 'left').quantityOrdered_y
    
    #ProductTotalReturned
    productTotalReturned = np.zeros(df_.shape[0])
    
    previousID = None
    
    returnDic = {}
    
    for row in df_.itertuples(): #iterate through dataFrame: row[0] = index, row[1] = productId, row[2] = orderDate
                                                           # row[3] = quantityOrdered, row[4] = quantityReturned
        if row[0] == 0:                                    # row[5] = returnDateTime
            
            #update return dictionary if this product is returned
            if row[4] != None:
                if row[5] in returnDic:
                    returnDic[row[5]] += row[4]
                else:
                    returnDic[row[5]] = row[4]

            previousID = row[1]
            
        elif (previousID == row[1]):
            
            #update return dictionary if this product is returned
            if row[4] != None:
                if row[5] in returnDic:
                    returnDic[row[5]] += row[4]
                else:
                    returnDic[row[5]] = row[4]
            
            #add returned products to new dictionary if known
            known = {k: v for k, v in returnDic.items() if k <= row[2]}
            productTotalReturned[row[0]] = sum(known.values())
            
            #update the dictionary by removing the returns which are now known
            returnDic = {k: v for k, v in returnDic.items() if k > row[2]}
                        
            previousID = row[1]
            
        else:
            returnDic = {} #new productId, hence empty the return dictionary
            
            #update return dictionary if this product is returned
            if row[4] != None:
                if row[5] in returnDic:
                    returnDic[row[5]] += row[4]
                else:
                    returnDic[row[5]] = row[4]
                    
            previousID = row[1]
    
    df_['productTotalReturned'] = productTotalReturned
    pivot = df_.groupby(by = ['productId','orderDate']).productTotalReturned.sum().groupby('productId').cumsum()
    productTotalReturned = df_.merge(pivot, 
                                left_on=['productId','orderDate'], 
                                right_index=True, 
                                how = 'left').productTotalReturned_y
     
    #Add new columns to dataFrame    
    df['productOrderCount0'] = productOrderCount
    df['productTotalCount0'] = productTotalCount
    df['productTotalReturned0'] = productTotalReturned
    df['productReturnFraction0'] = productTotalReturned / productTotalCount
    
    return(df)

In [35]:
def addProductColumnsX(df,X):
    """
    Function to add 4 columns: productOrderCountX, productTotalCountX, productTotalReturnedX and productReturnFractionX.
    Input: dataFrame with columns: 'productId','orderDate','productOrderCount','productTotalCount','productTotalReturned'
    """
    df = df.sort_values(by = ['productId','orderDate'], ascending = [True, False]) #reverse ordering on Orderdate!
    df = df.reset_index(drop = True)
    
    df_ = df[['productId','orderDate','productOrderCount0','productTotalCount0','productTotalReturned0']]
    #            row[1]       row[2]        row[3]               row[4]                 row[5]    
    
    df_['orderDateX'] = df_['orderDate'] + timedelta(X)
    #      row[6]

    knownProductInfo = np.zeros((df_.shape[0],3))
    
    previousID = None
    previousMaxDate = None
    
    dic = {}
    
    for row in df_.itertuples(): #iterate  
                                                                  
        if row[0] == 0:                                          
            
            knownProductInfo[[row[0]]] = (row[3],row[4],row[5]) 
            
            dic[row[2]] = (row[3],row[4],row[5])

            previousMaxDate = row[2]
            previousID = row[1]
            
        elif (previousID == row[1]):
            
            if row[6] >= previousMaxDate:
                dic[row[2]] = (row[3],row[4],row[5])
                knownProductInfo[[row[0]]] = dic[max(dic)]
            else:
                dic[row[2]] = (row[3],row[4],row[5])
                dic = {k: v for k, v in dic.items() if k <= row[6]}
                
                knownProductInfo[[row[0]]] = dic[max(dic)]
                previousMaxDate = max(dic)
                 
            previousID = row[1]
            
        else:
            dic = {} #new productId -> empty the dictionary
            
            knownProductInfo[[row[0]]] = (row[3],row[4],row[5])
            dic[row[2]] = (row[3],row[4],row[5])
                    
            previousMaxDate = row[2]
            previousID = row[1]

    df['productOrderCountX'] = knownProductInfo[:,0]
    df['productTotalCountX'] = knownProductInfo[:,1]
    df['productTotalReturnedX'] = knownProductInfo[:,2]
    df['productReturnFractionX'] = knownProductInfo[:,2] / knownProductInfo[:,1]
    
    #Reverse to natural order
    df = df.sort_values(by = ['productId','orderDate'], ascending = [True, True])
    df = df.reset_index(drop = True)
    
    return(df)

In [127]:
def addSellerColumns0(df):
    """
    Function to add 4 columns: 
    Input: dataFrame with columns: 'sellerId','orderDate','quantityOrdered','partnerSellingMonths'
    """
    df = df.sort_values(by = ['sellerId','orderDate'])
    df = df.reset_index(drop = True)
    
    df_ = df[['sellerId','orderDate','quantityOrdered','partnerSellingMonths']]
    
    firstOrder = df_.groupby('sellerId').orderDate.min()
    df_['firstOrder'] = df_.merge(firstOrder,
                                  left_on = 'sellerId',
                                  right_index = True,
                                  how = 'left').orderDate_y
    df_['daysFirstOrder'] = (df_['orderDate'] - df_['firstOrder']).dt.days + 1
    
    pivot = df_.groupby(['sellerId','orderDate']).quantityOrdered.count().groupby('sellerId').cumsum()
    sellerTotalCount = df_.merge(pivot, 
                                left_on=['sellerId','orderDate'], 
                                right_index=True, 
                                how = 'left').quantityOrdered_y
    
    df['sellerDailyOrders0'] = np.log(sellerTotalCount / df_['daysFirstOrder'])
    
    return df

In [128]:
def addSellerColumnsX(df,X):
    """
    Function to add 4 columns: 
    Input: dataFrame with columns: 'sellerId','orderDate','quantityOrdered','partnerSellingMonths'
    """
        
    df = df.sort_values(by = ['sellerId','orderDate'], ascending = [True, False]) #reverse ordering orderdate!
    df = df.reset_index(drop = True)

    df_ = df[['sellerId','orderDate','sellerDailyOrders0']]
    #            row[1]       row[2]        row[3]        

    df_['orderDateX'] = df_['orderDate'] + timedelta(X)
    #      row[4]

    knownSellerInfo = np.zeros(df_.shape[0])

    previousID = None
    previousMaxDate = None

    dic = {}

    for row in df_.itertuples(): #iterate  

        if row[0] == 0:                                          

            knownSellerInfo[[row[0]]] = row[3]

            dic[row[2]] = row[3]

            previousMaxDate = row[2]
            previousID = row[1]

        elif (previousID == row[1]):

            if row[4] >= previousMaxDate:
                dic[row[2]] = row[3]
                knownSellerInfo[[row[0]]] = dic[max(dic)]
            else:
                dic[row[2]] = row[3]
                dic = {k: v for k, v in dic.items() if k <= row[4]}

                knownSellerInfo[[row[0]]] = dic[max(dic)]
                previousMaxDate = max(dic)

            previousID = row[1]

        else:
            dic = {} #new productId -> empty the dictionary

            knownSellerInfo[[row[0]]] = row[3]
            dic[row[2]] = row[3]

            previousMaxDate = row[2]
            previousID = row[1]

    df['sellerDailyOrdersX'] = knownSellerInfo

    #Reverse to natural order
    df = df.sort_values(by = ['sellerId','orderDate'], ascending = [True, True])
    df = df.reset_index(drop = True)

    return df

#### TESTING

In [37]:
#df.loc[(df['productId'] == 9300000000762808) | (df['productId'] == 9200000101821794)]
#df.loc[(df['productId'] == 9200000105019922)]
#[['productId','orderDate','returnDateTime','quantityOrdered','quantityReturned']]
#[['caseDays','returnDays','cancellationDays','actualDeliveryDays']]

# t0 = time.time()
# a = addProductColumns(df)
# b = addProductColumnsX(a,2)
# t1 = time.time()
# total = t1-t0
# print(total)

In [140]:
# df_ = df.loc[(df['productId'] == 9300000000762808) | (df['productId'] == 9200000101821794)]
df_ = df.loc[(df['productId'] == 9200000105019922)]
# df_ = df.loc[df['sellerId'] == 1003411]
df_ = df_[['productId','orderDate','returnDateTime','quantityOrdered','quantityReturned',
           'caseDays','returnDays','cancellationDays','actualDeliveryDays','onTimeDelivery',
           'transporterCodeGeneral','shipmentDays']]
# df_ = df_[['productId','orderDate','productTotalCount','productOrderCount','productTotalReturned']]
# df_ = df_[['sellerId','orderDate','quantityOrdered','partnerSellingMonths']]

#df_ = addProductColumns(df_)
#df_ = addProductColumnsX(df_,2)
df_ = addProductColumns(df_,2)
df_

<ipython-input-135-87b2216d9be3>:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['productTotalReturned'] = productTotalReturned
<ipython-input-126-2e46c60816db>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['orderDateX'] = df_['orderDate'] + timedelta(X)


,productId,orderDate,returnDateTime,quantityOrdered,quantityReturned,caseDays,returnDays,cancellationDays,actualDeliveryDays,onTimeDelivery,transporterCodeGeneral,shipmentDays,productOrderCount0,productTotalCount0,productTotalReturned0,productReturnFraction0,productOrderCountX,productTotalCountX,productTotalReturnedX,productReturnFractionX
0,9200000105019922,2019-07-05,2019-07-09,1,1.0,NaN,4.0,NaN,3.0,True,POSTNL,1.0,1,1,0.0,0.000000,1.0,1.0,0.0,0.000000
1,9200000105019922,2019-07-30,NaT,1,NaN,NaN,NaN,NaN,1.0,True,POSTNL,0.0,5,5,1.0,0.200000,16.0,16.0,1.0,0.062500
2,9200000105019922,2019-07-30,NaT,1,NaN,NaN,NaN,NaN,NaN,NaN,BRIEF,1.0,5,5,1.0,0.200000,16.0,16.0,1.0,0.062500
3,9200000105019922,2019-07-30,NaT,1,NaN,NaN,NaN,NaN,NaN,NaN,BRIEF,1.0,5,5,1.0,0.200000,16.0,16.0,1.0,0.062500
4,9200000105019922,2019-07-30,NaT,1,NaN,NaN,NaN,NaN,1.0,True,POSTNL,0.0,5,5,1.0,0.200000,16.0,16.0,1.0,0.062500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,9200000105019922,2020-12-08,NaT,1,NaN,NaN,NaN,NaN,1.0,True,POSTNL,0.0,565,577,31.0,0.053726,566.0,578.0,31.0,0.053633
565,9200000105019922,2020-12-09,NaT,1,NaN,NaN,NaN,NaN,NaN,NaN,BRIEF,0.0,566,578,31.0,0.053633,567.0,580.0,31.0,0.053448
566,9200000105019922,2020-12-11,NaT,2,NaN,NaN,NaN,NaN,1.0,True,POSTNL,0.0,567,580,31.0,0.053448,567.0,580.0,31.0,0.053448
567,9200000105019922,2020-12-15,NaT,1,NaN,NaN,NaN,NaN,NaN,NaN,BRIEF,0.0,568,581,31.0,0.053356,569.0,582.0,31.0,0.053265


### Models